In [9]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np
import unidecode
import contractions
from word2number import w2n

df = pd.read_csv('questions.csv')
df.head()
# df.tail()


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [10]:
df[df['is_duplicate'] == 1].head()
df.count()
df['is_duplicate'].sum()

149306

In [11]:
import nltk
# nltk.download()


In [12]:
from sklearn.model_selection import train_test_split
y = df['is_duplicate']
# dft = df.drop('is_duplicate', axis=1)
dft = df
X_train, X_test, y_train, y_test = train_test_split(dft, y, test_size=0.98, random_state=42)

In [13]:
X_train.count()

id              8087
qid1            8087
qid2            8087
question1       8087
question2       8087
is_duplicate    8087
dtype: int64

In [14]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import string

snowball = SnowballStemmer(language='english')
stop_words = stopwords.words('english')
# print(f'stop words: {stop_words}')
punct_table = str.maketrans('', '', string.punctuation)



In [19]:
def clean_text(text):
    try:
        if pd.isna(text):
            text = ''
    
        text = unidecode.unidecode(text)
        text = contractions.fix(text)
        text = text.lower()

        words = word_tokenize(text)
        words = [w.translate(punct_table) for w in words]
        words = [w for w in words if not w in stop_words]
        words = [snowball.stem(w) for w in words]
        text = ' '.join(words)

    except:
        print(f'error: |{text}|')
        raise ValueError(f'failed to parse value: {text}')
    
    return text

In [20]:
dft = X_train
df_clean1 = df['question1'].apply(clean_text)
df_clean1.head()

0            step step guid invest share market india 
1                   stori kohinoor  kohinoor  diamond 
2              increas speed internet connect use vpn 
3                                   mental lone  solv 
4    one dissolv water quik sugar  salt  methan car...
Name: question1, dtype: object

In [21]:
df_clean2 = df['question2'].apply(clean_text)
df_clean2.head()

0                  step step guid invest share market 
1    would happen indian govern stole kohinoor  koh...
2                     internet speed increas hack dns 
3       find remaind  math  23  24   math  divid 2423 
4                        fish would surviv salt water 
Name: question2, dtype: object